In [1]:
import pandas as pd
import numpy as np
import vcf
import h5py
import csv
import os
from random import seed, random

# Generate ClinVar Data for Semantic Zoom

We will use a clinvar *.vcf file as an input to generate two following files:
1. HiGlass' scalable BED file: Depending on the zoom levels, this file contain selective variant information.
2. HiGlass' multivec file: This file contains the frequency of variant information for every binned region along genomic axis.

These two files can be used together in a single Gosling.js track to allow semantic zooming:
- When zoomed out, show density information
- When zoomed in, show detailed information


clodius aggregate bedfile --chromsizes-filename hg38.txt --delimiter $'\t' --importance-column 6 --max-per-tile 80 clinvar_20200824.bed 

python manage.py ingest_tileset --filename data/clinvar_20200824_v1.beddb --filetype beddb --datatype bedlike --uid clinvar_20200824_v1

## Create The Two Files

In [6]:
vcf_file = os.path.join('data', 'clinvar.vcf')
bed_file = os.path.join('data', 'clinvar.bed')
mv5_file = os.path.join('data', 'density.hdf5')
chr_file = os.path.join('data', 'hg38_full.txt')

In [7]:
# We are interested only in the following information
CHR_FILTER = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y']

SIGNIFICANCE_FILTER = [
    'Benign',
    'Benign/Likely_benign',
    'Likely_benign',
    'Uncertain_significance',
    'Likely_pathogenic',
    'Pathogenic/Likely_pathogenic',
    'Pathogenic',
    'risk_factor',
    'Conflicting_interpretations_of_pathogenicity'
]

GOLD_STARS_FILTER = [
    'criteria_provided,_multiple_submitters,_no_conflicts',
    'criteria_provided,_single_submitter',
    'criteria_provided,_conflicting_interpretations',
    'no_assertion_criteria_provided', 
    'reviewed_by_expert_panel', 
    'practice_guideline', 
    'no_interpretation_for_the_single_variant', 
    'no_assertion_provided'
]

In [8]:
df = pd.read_csv(chr_file, '\t', header=None, names=['chr', 'length'])

df = df[df.chr.isin(['chr' + c for c in CHR_FILTER])]

TOTAL_CHROM_LEN = df.length.sum()

# l = df[df.chr == 'chr1'].length
# l

In [9]:
# This code is based on the following public codes:
# https://github.com/higlass/higlass-clinvar/blob/master/scripts/extract_clinvar_data.py
# https://github.com/higlass/higlass-sequence/blob/master/scripts/fasta_to_hdf5.py

"""
Extracted Keys
"""

"""
'AF_ESP'
'AF_EXAC'
'AF_TGP'
'ALLELEID'
'ALT'
'CHROM'
'CLNDISDB'
'CLNDISDBINCL'
'CLNDN'
'CLNDNINCL'
'CLNHGVS'
'CLNREVSTAT'
'CLNSIG'
'CLNSIGCONF'
'CLNSIGINCL'
'CLNVC'
'CLNVCSO'
'CLNVI'
'DBVARID'
'FILTER_PASS'
'GENEINFO'
'ID'
'MC'
'ORIGIN'
'POS'
'QUAL'
'REF'
'RS'
'SSR'
'altlen'
'is_snp'
'numalt'
"""

vcf_data = vcf.Reader(open(vcf_file, 'r'))
mv5_data = h5py.File(mv5_file, "w")

seed(1)

output = []
test = []
density_dict = { c : np.zeros((df[df.chr == ('chr' + c)].length.sum(), len(SIGNIFICANCE_FILTER))) for c in CHR_FILTER}

for record in vcf_data:
    #print(record.INFO)    
    if not 'CLNSIG' in record.INFO:
        continue

    if not 'CLNREVSTAT' in record.INFO:
        continue

    gold_stars_str = ','.join(record.INFO['CLNREVSTAT'])


    if not gold_stars_str in GOLD_STARS_FILTER:
        test.append(gold_stars_str)
        continue

    significance = record.INFO['CLNSIG'][0]
    if not significance in SIGNIFICANCE_FILTER:
        #test.append(significance)
        continue

    gold_stars = 0
    if gold_stars_str == 'practice_guideline':
        gold_stars = 4
    elif gold_stars_str == 'reviewed_by_expert_panel':
        gold_stars = 3
    elif gold_stars_str == 'criteria_provided,_multiple_submitters,_no_conflicts':
        gold_stars = 2
    elif gold_stars_str in ['criteria_provided,_single_submitter', 'criteria_provided,_conflicting_interpretations']:
        gold_stars = 1

    disease_name = record.INFO['CLNDN'][0] if "CLNDN" in record.INFO else "."

    if disease_name == "not_provided":
        disease_name = "."

    if not record.CHROM in CHR_FILTER:
        continue

    #print(record)
    
    """
    BED
    """
    data_clean = {
        'chr': 'chr' + record.CHROM, 
        'start': +record.POS, 
        'end': +record.POS+1, 
        'ref': record.REF,
        'alt': record.ALT[0].sequence if record.ALT[0] else ".",
        'importance': gold_stars + random(),
        'gold_stars': gold_stars,
        'significance': significance,
        'significance_conf': ','.join(record.INFO['CLNSIGCONF']) if "CLNSIGCONF" in record.INFO else ".",
        'variant_type': record.INFO['CLNVC'] if "CLNVC" in record.INFO else ".",
        'origin': record.INFO['ORIGIN'][0] if "ORIGIN" in record.INFO else ".",
        'molecular_consequence': record.INFO['MC'][0] if "MC" in record.INFO else ".",
        'disease_name': disease_name,
        'hgvs': record.INFO['CLNHGVS'][0].replace("%3D", "=") if "CLNHGVS" in record.INFO else ".",
        #'database': record.INFO['CLNDISDB'][0] if "CLNDISDB" in record.INFO else ".",        
    }
    
    output.append(data_clean)
    
    """
    MV5
    """    
    # column is significance
    column_index = SIGNIFICANCE_FILTER.index(significance)
    
    # Add frequency by 1
    density_dict[record.CHROM][record.POS, column_index] = density_dict[record.CHROM][record.POS, column_index] + 1
    
myset = set(test)
print(myset)

print(output[0])
print(output[33])

print("Writing to BED file")
headers = output[0].keys()
with open(bed_file, 'w') as opf:
    myWriter = csv.DictWriter(opf, delimiter='\t', fieldnames=headers)
    for row in output:
        myWriter.writerow(row)

print("Writing to hdf5 file")
for c in CHR_FILTER:
    print("chr" + c)
    mv5_data.create_dataset(('chr' + c), (df[df.chr == ('chr' + c)].length.sum(), len(SIGNIFICANCE_FILTER)), "i", data=density_dict[c], compression='gzip')

mv5_data.close()

NameError: name 'SIGNIFICANCE_FILTER' is not defined

Then, run the following commands:

```sh
clodius aggregate bedfile \
    --chromsizes-filename hg38_full.txt \
    --delimiter $'\t' \
    --importance-column 6 \
    --max-per-tile 80 \
    clinvar.bed

clodius aggregate multivec \
    --chromsizes-filename hg38_full.txt \
    --starting-resolution 1 \
    density.hdf5

python manage.py ingest_tileset \
    --filename clinvar.bed.beddb \
    --filetype beddb \
    --datatype bedlike \
    --uid clinvar
```